# Assignment 1: Web Scraping

## Objective

Data scientists often need to crawl data from websites and turn the crawled data (HTML pages) to structured data (tables). Thus, web scraping is an essential skill that every data scientist should master. In this assignment, you will learn the followings:


* How to download HTML pages from a website?
* How to extract relevant content from an HTML page? 

Furthermore, you will gain a deeper understanding of the data science lifecycle.

**Requirements:**

1. Please use [pandas.DataFrame](http://pandas.pydata.org/pandas-docs/stable/generated/pandas.DataFrame.html) rather than spark.DataFrame to manipulate data.

2. Please use [BeautifulSoup](https://www.crummy.com/software/BeautifulSoup/) rather than [lxml](http://lxml.de/) to parse an HTML page and extract data from the page.

3. Please follow python code style (https://www.python.org/dev/peps/pep-0008/). If TA finds your code hard to read, you will lose points. This requirement will stay for the whole semester.

## Preliminary

If this is your first time to write a web scraper, you need to learn some basic knowledge of this topic. I found that this is a good resource: [Tutorial: Web Scraping and BeautifulSoup](https://www.dataquest.io/blog/web-scraping-beautifulsoup/). 

Please let me know if you find a better resource. I'll share it with the other students.

## Overview

Imagine you are a data scientist working at SFU. Your job is to extract insights from SFU data to answer questions. 

In this assignment, you will do two tasks. Please recall the high-level data science lifecycle below. I suggest that when doing this assignment, please remind yourself of what data you collected and what questions you tried to answer.


<img src="lifecycle.png" width="500">
<center><h4>Figure 1. Data Science Lifecycle</h4></center>

## Task 1: SFU CS Faculty Members

Sometimes you don't know what questions to ask. No worries. Start collecting data first. 

In Task 1, your job is to write a web scraper to extract the faculty information from this page: [https://www.sfu.ca/computing/people/faculty.html](https://www.sfu.ca/computing/people/faculty.html).




### (a) Crawl Web Page

A web page is essentially a file stored in a remote machine (called web server). Please write code to download the HTML page and save it as a text file ("csfaculty.html").

In [7]:
import urllib.request
urllib.request.urlretrieve ("https://www.sfu.ca/computing/people/faculty.html.", "csfaculty.html")

('csfaculty.html', <http.client.HTTPMessage at 0x7fd094535cc0>)

### (b) Extract Structured Data

Please write code to extract relevant content (name, rank, area, profile, homepage) from "csfaculty.html" and save them as a CSV file (like [faculty_table.csv](./faculty_table.csv)). 

In [8]:
from bs4 import BeautifulSoup
import pandas as pd
import re

#Defining Regular Expressions
area_regex = re.compile('Area:(.*)')
prof_regex = re.compile('^/computing.*')
sfu_home = 'http://www.sfu.ca'

#To decode name and rank information
def fetch_namerank(elem):
    if elem.h4 is not None:
        h4elem = elem.h4.text
        namerank = h4elem.split(",")
        nam = namerank[0].title()
        cleanrank = namerank[1].split("\n")
        ran = cleanrank[0].strip().title()
    return nam,ran

#To decode area information using regex match with p text
def fetch_area(peach):
    area_re = re.search(area_regex, peach.p.text)
    if area_re is None:
        area_re = re.search(area_regex, peach.text)        
    area_f = area_re.group(1).strip()
    return area_f

#Using Regex to check partial links. Some profiles are provided with full links
def partial_link_check(link):
    if re.search(prof_regex, link):
        full_link = sfu_home + link
    else:
        full_link = link
    return full_link

#Profile Homepage extraction
def prof_homepage(pelem):
    prof = ''
    home = ''
    # Searching for a tags with href
    ahref = pelem.find_all('a',href=True)
    if len(ahref) >= 1:
        prof = str(partial_link_check(ahref[0].get('href')))
        if len(ahref) >= 2:
            home = str(partial_link_check(ahref[1].get('href')))
    return prof,home

def main(file):
    #Opening and Reading the file
    file = open(file,'r')
    content = file.read()

    #Creation of pandas dataframe to create output results
    df_faculty = pd.DataFrame(columns=['name','rank','area','profile','homepage'])

    # Parsing the HTML file
    html_soup = BeautifulSoup(content, 'html.parser')

    #Extracting only div with class attribute text
    faculty_container = html_soup.find_all('div', class_ = "parsys_column cq-colctrl-lt0 people faculty-list")

    #Processing of each element
    for parent in faculty_container:    
        child = parent.find_all('div', class_ = "text")
        for each in child:
            # Name and Rank information
            name,rank = fetch_namerank(each)
            
            #Area information
            area = fetch_area(each)
            
            #Profile and homepage information fetched from p tag
            ptag = each.find_all('p')
            profile = ''
            homepage = ''
            # All the profile information are present in first 3 records. Hence the check
            if len(ptag) > 1:
                p = ptag[1]
                profile,homepage = prof_homepage(p)
                if profile == '':
                    p = ptag[2]
                    profile,homepage = prof_homepage(p)
            else:
                p = ptag[0]
                profile,homepage = prof_homepage(p)    
                
            # Appending to pandas dataframe
            df_faculty = df_faculty.append({'name':name,'rank':rank,'area':area,'profile':profile,'homepage':homepage}, ignore_index=True)

    #Writing to csv        
    df_faculty.to_csv('faculty_table.csv',index=False)

if __name__ == '__main__':
    filename = 'csfaculty.html'
    main(filename)

### (c) Interesting Finding

Note that you don't need to do anything for Task 1(c). The purpose of this part is to give you some sense about how to leverage exploratory data analysis (EDA) to come up with interesting questions about the data. EDA is an important topic in data science; you will  learn it soon from this course. 


First, please install [dataprep](http://dataprep.ai).
Then, run the cell below. 
It shows a bar chart for every column. What intersting findings can you get from these visualizations? 

In [9]:
from dataprep.eda import plot
import pandas as pd

df = pd.read_csv("faculty_table.csv")
plot(df)

GridBox(id='1637', ...)

Below are some examples:

**Finding 1:** Professor# (29) is more than 3x larger than Associate Professor# (9). 

**Questions:** Why did it happen? Is it common in all CS schools in Canada? Will the gap go larger or smaller in five years? What actions can be taken to enlarge/shrink the gap?


**Finding 2:** Homepage has 20.3% missing values. 

**Questions:** Why are there so many missing values? Is it because many faculty do not have their own homepages or do not add their homepages to the school page? What actions can be taken to avoid this to happen in the future? 

## Task 2: Age Follows Normal Distribution?

In this task, you start with a question and then figure out what data to collect.

The question that you are intersted in is `Does SFU CS faculty age follow a normal distribution?`

To estimate the age of a faculty member, you can collect the year in which s/he graduates from a university (`gradyear`) and then estimate `age` using the following equation:

$$age \approx 2020+23 - gradyear$$

For example, if one graduates from a university in 1990, then the age is estimated as 2020+23-1990 = 53. 



### (a) Crawl Web Page

You notice that faculty profile pages contain graduation information. For example, you can see that Dr. Jiannan Wang graduated from Harbin Institute of Technology in 2008 at [http://www.sfu.ca/computing/people/faculty/jiannanwang.html](http://www.sfu.ca/computing/people/faculty/jiannanwang.html). 


Please write code to download the 64 profile pages and save each page as a text file. 

In [10]:
import requests
import pandas as pd
import os

# Reading the file faculty_table.csv
df_prof = pd.read_csv("faculty_table.csv")

#Looping through dataframe
for idx in df_prof.index:
    # Fetching name to store it along with path
    name = df_prof['name'][idx]
    # Taking the profile link for each name
    profile = df_prof['profile'][idx]
    # Setting the file naming conventions
    pathname = './profpage/'+ str(idx) + '-' + name + '.txt'
    #Making the directory with filename
    os.makedirs(os.path.dirname(pathname), exist_ok=True)
    #Perform request-reply for profile link
    response = requests.get(profile)
    #Open and store the reply data into the filename
    f = open(pathname, 'w')
    f.write(response.text)
    f.close()

### (b) Extract Structured Data

Please write code to extract the earliest graduation year (e.g., 2008 for Dr. Jiannan Wang) from each profile page, and create a csv file like [faculty_grad_year.csv](./faculty_grad_year.csv). 

In [11]:
import pandas as pd
import unicodedata
import itertools

# Function to extract graduation years alone and take the minimal year that shall be the earliest
def fetch_grades(etag):
    lst= []
    yr = ''
    if len(etag) > 0:
        for tg in etag:
            lst.append(str(tg.text))
    tokens = [i.replace("."," ").split() for i in lst]
    merged = list(itertools.chain.from_iterable(tokens))
    if len(merged) > 0:
        digits = [int(item) for item in merged if item.isdigit()]
        if digits:
            yr = min(digits)
    return yr
    

# Main function to extract name and graduation year 
def get_grad_year():
    
    # Read the faculty_table file
    df_prof = pd.read_csv("faculty_table.csv")
    
    #Creation of pandas dataframe to create output results
    df_gradyear = pd.DataFrame(columns=['name','gradyear'])
    
    #Loop through each profile page to collect information
    for ids in df_prof.index:
        name = df_prof['name'][ids]
        profile = df_prof['profile'][ids]
        pathname = './profpage/'+ str(ids) + '-' + name + '.txt'
        profile_fil = open(pathname, 'r')
        profile_det = profile_fil.read()
        
        # Parsing the HTML file
        html_prof = BeautifulSoup(profile_det, 'html.parser')
        
        # To obtain names from profile page
        title = html_prof.find_all('div', class_ = "title section")
        for names in title:
            final_name = names.h1.text
        
        # To obtain earliest graduate year
        edu_parent = html_prof.find('h2', text='Education').parent
        tagp = edu_parent.find_all("p")
        year = fetch_grades(tagp)
        
        if year == '':
            tagl = edu_parent.find_all("ul")
            year = fetch_grades(tagl)
        
        # Loading the information into dataframe
        df_gradyear = df_gradyear.append({'name':final_name,'gradyear':year},ignore_index=True)
        
    #Writing to csv        
    df_gradyear.to_csv('faculty_grad_year.csv',index=True)


# Function to extract earliest graduation year
get_grad_year()

### (c) Interesting Finding

Similar to Task 1(c), you don't need to do anything here. Just look at different visualizations w.r.t. age and give yourself an answer to the question: `Does SFU CS faculty age follow a normal distribution?`

In [12]:
from dataprep.eda import plot
import pandas as pd

df = pd.read_csv("faculty_grad_year.csv")
df["age"] = 2020+23-df["gradyear"]

plot(df, "age")

Tabs(id='2059', ...)

## Submission

Complete the code in this [notebook](https://github.com/sfu-db/bigdata-cmpt733/blob/master/Assignments/A1/A1.ipynb), and submit it to the CourSys activity `Assignment 1`.

<hr/>
## <span style="color:red">Bonus: Contribute to dataprep</span>

**Total Bonus: 0.2 + 0.3 = 0.5**

1. If you create an issue (i.e., report a bug or request a feature) at [link1](https://github.com/sfu-db/dataprep/issues) or [link2](https://github.com/sfu-db/DataConnectorConfigs/issues), you will get **0.2** bonus points.

2.  Creating more issues will *not* give you more bonus points, but you are encouraged to do so for learning more.

2. If you send a pull request (i.e., fix a bug or implement a feature or add data connector for a new website) at [link1](https://github.com/sfu-db/dataprep/pulls) or [link2](https://github.com/sfu-db/DataConnectorConfigs/pulls) and the pull request gets merged into the repo, you will get **0.3** bonus points. 

4.  Sending more pull requests will *not* give you more bonus points, but you are encouraged to do so for learning more.

5. These bonus points will be directly added to your final grade.






**How to Submit**
* Submit github link(s) to the CourSys activity `Bonus`.
* Due on `March 15, 2020`

If you love dataprep, please support it by simply clicking the **<span style="color:red">Star</span>** on [Github](https://github.com/sfu-db/dataprep).
<img src="dataprep-star.png" width="1000">